1. Let's import the modules we'll need to start off with.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import glob
from datetime import date

2. Now let's perform the following steps:
    - Creata a variable **files**, where we'll *glob* together the dozen or so CSV files we have.
    - Create an empty **df_list**, and then append each CSV file's data to it using a *for* loop
    - Concatenate the **df_list** into a single dataframe **df**.

In [2]:
files = glob.glob("JC-2016**-citibike-tripdata.csv")
df_list = []

for file in files:
    data = pd.read_csv(file)
    df_list.append(data)

df = pd.concat(df_list)

3. Let's check out the length of the data and then the first five rows of our **df** dataset to get an idea of what we have.

In [3]:
print(len(df))
print(df.head)

247584
<bound method NDFrame.head of        Trip Duration           Start Time            Stop Time  \
0                362  2016-01-01 00:02:52  2016-01-01 00:08:54   
1                200  2016-01-01 00:18:22  2016-01-01 00:21:42   
2                202  2016-01-01 00:18:25  2016-01-01 00:21:47   
3                248  2016-01-01 00:23:13  2016-01-01 00:27:21   
4                903  2016-01-01 01:03:20  2016-01-01 01:18:24   
...              ...                  ...                  ...   
15109            557  2016-12-31 23:10:16  2016-12-31 23:19:33   
15110           2749  2016-12-31 23:29:39  2017-01-01 00:15:29   
15111            173  2016-12-31 23:44:37  2016-12-31 23:47:31   
15112           2424  2016-12-31 23:44:50  2017-01-01 00:25:14   
15113           2419  2016-12-31 23:44:50  2017-01-01 00:25:10   

       Start Station ID Start Station Name  Start Station Latitude  \
0                  3186      Grove St PATH               40.719586   
1                  3186      G

4. Now let's start our data cleaning. First, let's see if, and how many of our 247,584 rows have null or missing data.

In [4]:
print(df.isna().sum())

Trip Duration                  0
Start Time                     0
Stop Time                      0
Start Station ID               0
Start Station Name             0
Start Station Latitude         0
Start Station Longitude        0
End Station ID                 0
End Station Name               0
End Station Latitude           0
End Station Longitude          0
Bike ID                        0
User Type                    380
Birth Year                 18999
Gender                         0
dtype: int64


5. Now, let's let **df** be our dataset with the missing or null data *drop*ped. Let's check that by printing the tally of rows with missing data again.

In [5]:
df = df.dropna()
print(df.isna().sum())

Trip Duration              0
Start Time                 0
Stop Time                  0
Start Station ID           0
Start Station Name         0
Start Station Latitude     0
Start Station Longitude    0
End Station ID             0
End Station Name           0
End Station Latitude       0
End Station Longitude      0
Bike ID                    0
User Type                  0
Birth Year                 0
Gender                     0
dtype: int64


Indeed, no rows of missing or null data, which helps clean our data tremendously

6. Let's see how many rows our dataset is now.

In [6]:
print(len(df))

228205


Our dataset is still quite large at 228,205 rows.

7. Let's see what our column names are.

In [7]:
print(df.columns)

Index(['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type',
       'Birth Year', 'Gender'],
      dtype='object')


8. Now, let's *describe* **df** to see if we can see any oddities or potential outliers in our data.

In [8]:
print(df.describe())

       Trip Duration  Start Station ID  Start Station Latitude  \
count   2.282050e+05     228205.000000           228205.000000   
mean    7.423686e+02       3207.201240               40.723299   
std     3.699906e+04         27.004405                0.008149   
min     6.100000e+01       3183.000000               40.692640   
25%     2.420000e+02       3186.000000               40.717732   
50%     3.700000e+02       3202.000000               40.721525   
75%     6.110000e+02       3211.000000               40.727596   
max     1.632981e+07       3426.000000               40.752559   

       Start Station Longitude  End Station ID  End Station Latitude  \
count            228205.000000   228205.000000         228205.000000   
mean                -74.046623     3203.900107             40.722738   
std                   0.011176       55.399628              0.007904   
min                 -74.096937      173.000000             40.692216   
25%                 -74.050656     3186.00000

Several things stick out at first glance that we'll want to investigate further:
1. There is a minimum Birth Year of 1900. While it is technically possible to have a 116-year-old person still living, it's quite unlikely for someone that old still be able to ride a bicycle.
2. There is quite a long maximum trip duration, over 16.32981 million seconds, which would equate to approximately 272,163 minutes, or about 4,536 hours, or about 189 days. That seems like one long bike trip!

8. Let's investigate this apparent Birth Year anomaly. Let's print out the unique Birth Year values.

In [9]:
print(df['Birth Year'].unique())

[1964. 1962. 1984. 1988. 1980. 1990. 1993. 1976. 1992. 1991. 1968. 1975.
 1982. 1981. 1986. 1966. 1972. 1977. 1979. 1987. 1983. 1957. 1956. 1960.
 1985. 1969. 1989. 1950. 1978. 1961. 1996. 1970. 1974. 1971. 1973. 1965.
 1963. 1994. 1949. 1955. 1967. 1951. 1959. 1954. 1953. 1958. 1944. 1941.
 1995. 1998. 1952. 1945. 1947. 1948. 1999. 1997. 1946. 1943. 2000. 1940.
 1934. 1942. 1937. 1900.]


9. It looks like there may not be a birth year prior to the 1930s decade, except for 1900, but let's narrow down this list to birth years to 1950 and before to get a closer look.

In [10]:
print(df['Birth Year'][df['Birth Year'] <= 1950].unique())

[1950. 1949. 1944. 1941. 1945. 1947. 1948. 1946. 1943. 1940. 1934. 1942.
 1937. 1900.]


Confirmed. 1900 is the only pre-1930s birth year, so we'll consider this an outlier. Most likely someone made a typo when purchasing a pass.

10. Let's drop this outlier by letting our **df** be equal to any rows with Birth Year being greater than or equal to 1930. Let's also confirm no 1900 birth year remains.

In [11]:
df = df[df['Birth Year'] >= 1930]
print(df['Birth Year'][df['Birth Year'] <= 1950].unique())

[1950. 1949. 1944. 1941. 1945. 1947. 1948. 1946. 1943. 1940. 1934. 1942.
 1937.]


11. Now let's investigate the apparent outlier of the Trip Duration column. First let's check out the mininum, maximum, and median values of Trip Duration.

In [12]:
print(df['Trip Duration'].min())
print(df['Trip Duration'].max())
print(df['Trip Duration'].median())

61
16329808
370.0


It seems that the 61-second minimum, and 370-second median make this 16,329,808-second maximum all that much more of an outlier. 

12. Let's print any rows that have a Trip Duration more than 1 million seconds to see if there are any other trip durations that are similarly outlying.

In [13]:
print(df[df['Trip Duration'] > 1000000])

       Trip Duration           Start Time            Stop Time  \
3233         2104123  2016-02-12 07:27:56  2016-03-07 15:56:40   
3269         2100551  2016-02-12 08:31:06  2016-03-07 16:00:18   
3371         2071209  2016-02-12 16:32:54  2016-03-07 15:53:03   
8903        16329808  2016-03-22 07:02:10  2016-09-27 07:05:38   
14746        1837255  2016-04-28 09:05:14  2016-05-19 15:26:09   
2894         1120971  2016-09-03 22:05:27  2016-09-16 21:28:18   
11356        1532001  2016-09-11 16:32:21  2016-09-29 10:05:42   
18510        4826890  2016-11-23 17:38:36  2017-01-18 14:26:46   

       Start Station ID    Start Station Name  Start Station Latitude  \
3233               3214      Essex Light Rail               40.712774   
3269               3184           Paulus Hook               40.714145   
3371               3183        Exchange Place               40.716247   
8903               3215           Central Ave               40.746730   
14746              3192    Liberty Light

It looks like there are several other Trip Durations greater than 1 million seconds, including one with over 4.8 million seconds. It's also interesting to note that all of these Trip Durations are "Subscriber" User Types. In looking at the accompanying documentation provided, these "Subscriber"s have annual memberships. One whole year is 31,536,000 seconds, so these apparent outlyers are actually sensible. This data is not dirty as first thought, so we'll leave this data alone.

Now let's shift gears and think about how we want to break our data up so that we can create a database schema for a PostGreSQL database. When looking at the columns earlier, it appears that there are multiple columns related to the stations, such as latitude, longitude, ID, and Name. This seems like the perfect candidate to break off into its own database.

13. First, let's identify which stations are used to start a bike trip while retaining that station's information. We'll subset this information into a new **start_stations** dataframe. We'll want to drop duplicates since we want a list of unique stations. While we're at it, let's rename the columns to make it easier to use later in our future database.

In [14]:
start_stations = df[['Start Station ID', 'Start Station Name', 'Start Station Latitude', 'Start Station Longitude']]
# print(start_stations)
start_stations = start_stations.drop_duplicates()
start_stations = start_stations.rename(columns={'Start Station ID':'station_id', 'Start Station Name': 'station_name', 'Start Station Latitude': 'station_lat', 'Start Station Longitude':'station_long'})
print(start_stations)
# print(len(start_stations))

       station_id             station_name  station_lat  station_long
0            3186            Grove St PATH    40.719586    -74.043117
3            3209             Brunswick St    40.724176    -74.050656
7            3211               Newark Ave    40.721525    -74.046305
8            3187                Warren St    40.721124    -74.038051
10           3183           Exchange Place    40.716247    -74.033459
11           3213           Van Vorst Park    40.718489    -74.047727
14           3193             Lincoln Park    40.724605    -74.078406
19           3194          McGinley Square    40.725340    -74.067622
20           3202             Newport PATH    40.727224    -74.033759
23           3196           Riverview Park    40.744319    -74.043991
27           3214         Essex Light Rail    40.712774    -74.036486
28           3195                  Sip Ave    40.730743    -74.063784
33           3207              Oakland Ave    40.737604    -74.052478
37           3199   

14. Let's repeat these exact steps for our end station data for an **end_stations**. Let's use the exact same column names as  **start_stations** so that we can concatenate these two dataframes together in the next step.

In [15]:
end_stations = df[['End Station ID', 'End Station Name', 'End Station Latitude', 'End Station Longitude']]
# print(end_stations)
end_stations = end_stations.drop_duplicates()
end_stations = end_stations.rename(columns={'End Station ID':'station_id', 'End Station Name': 'station_name', 'End Station Latitude': 'station_lat', 'End Station Longitude':'station_long'})
print(end_stations)
# print(len(end_stations))

      station_id                   station_name  station_lat  station_long
0           3209                   Brunswick St    40.724176    -74.050656
1           3213                 Van Vorst Park    40.718489    -74.047727
3           3203                  Hamilton Park    40.727596    -74.044247
8           3214               Essex Light Rail    40.712774    -74.036486
10          3187                      Warren St    40.721124    -74.038051
...          ...                            ...          ...           ...
1400        2004              6 Ave & Broome St    40.724399    -74.004704
5142         393              E 5 St & Avenue C    40.722992    -73.979955
9792        3277        Communipaw & Berry Lane    40.714358    -74.066611
245          405  Washington St & Gansevoort St    40.739323    -74.008119
5488         224          Spruce St & Nassau St    40.711464    -74.005524

[90 rows x 4 columns]


15. Now let's concatenate these dataframes into one **all_stations** dataframe. Then let's sort them into numerical order by station_id, and then let's drop any duplicate stations since we want unique stations.

In [16]:
all_stations = pd.concat([start_stations, end_stations])
all_stations = all_stations.sort_values(by='station_id')
all_stations = all_stations.drop_duplicates()
print(all_stations)

       station_id                  station_name  station_lat  station_long
7642          173            Broadway & W 49 St    40.760683    -73.984527
5488          224         Spruce St & Nassau St    40.711464    -74.005524
5439          225       W 14 St & The High Line    40.741951    -74.008030
1188          249       Harrison St & Hudson St    40.718710    -74.009001
7822          252  MacDougal St & Washington Sq    40.732264    -73.998522
...           ...                           ...          ...           ...
27614        3280                   Astor Place    40.719282    -74.071262
23085        3281           Leonard Gordon Park    40.745910    -74.057271
6194         3314            W 95 St & Broadway    40.793770    -73.971888
22692        3331       Riverside Dr & W 104 St    40.801343    -73.971146
573          3426                    JCBS Depot    40.709651    -74.068601

[90 rows x 4 columns]


16. Now let's clean up the **df** dataframe by eliminating the station information, except for the Start Station ID and End Station ID columns. In our upcoming database, we can perform joins using these columns to pull all the station information we need from our newly created **all_stations** dataframe.

In [17]:
df = df[['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID', 'End Station ID', 'Bike ID', 'User Type',
       'Birth Year', 'Gender']]
print(df.head())

   Trip Duration           Start Time            Stop Time  Start Station ID  \
0            362  2016-01-01 00:02:52  2016-01-01 00:08:54              3186   
1            200  2016-01-01 00:18:22  2016-01-01 00:21:42              3186   
2            202  2016-01-01 00:18:25  2016-01-01 00:21:47              3186   
3            248  2016-01-01 00:23:13  2016-01-01 00:27:21              3209   
6            445  2016-01-01 01:07:45  2016-01-01 01:15:11              3186   

   End Station ID  Bike ID   User Type  Birth Year  Gender  
0            3209    24647  Subscriber      1964.0       2  
1            3213    24605  Subscriber      1962.0       1  
2            3213    24689  Subscriber      1962.0       2  
3            3203    24693  Subscriber      1984.0       1  
6            3203    24510  Subscriber      1988.0       2  


17. Next, let's rename the **df** columns so that they will be easier to work with in our upcoming database.

In [18]:
df = df.rename(columns={'Trip Duration':'trip_duration_sec', 'Start Time':'start_time', 'Stop Time':'stop_time', 
                        'Start Station ID':'start_station_id', 'End Station ID':'end_station_id', 'Bike ID':'bike_id',
                        'User Type':'user_type','Birth Year':'birth_year', 'Gender':'gender'})
print(df.head())

   trip_duration_sec           start_time            stop_time  \
0                362  2016-01-01 00:02:52  2016-01-01 00:08:54   
1                200  2016-01-01 00:18:22  2016-01-01 00:21:42   
2                202  2016-01-01 00:18:25  2016-01-01 00:21:47   
3                248  2016-01-01 00:23:13  2016-01-01 00:27:21   
6                445  2016-01-01 01:07:45  2016-01-01 01:15:11   

   start_station_id  end_station_id  bike_id   user_type  birth_year  gender  
0              3186            3209    24647  Subscriber      1964.0       2  
1              3186            3213    24605  Subscriber      1962.0       1  
2              3186            3213    24689  Subscriber      1962.0       2  
3              3209            3203    24693  Subscriber      1984.0       1  
6              3186            3203    24510  Subscriber      1988.0       2  


18. Now, let's print the data type of each column to see if the data types are as expected. First with **df**, then with the **all_stations** dataset.

In [19]:
print(df.dtypes)
print(all_stations.dtypes)

trip_duration_sec      int64
start_time            object
stop_time             object
start_station_id       int64
end_station_id         int64
bike_id                int64
user_type             object
birth_year           float64
gender                 int64
dtype: object
station_id        int64
station_name     object
station_lat     float64
station_long    float64
dtype: object


19. Let's replace the numbers in the gender column as follows:
- 2: Female
- 1: Male
- 0: Unknown

In [20]:
df.gender = df.gender.replace(2, 'Female')
df.gender = df.gender.replace(1, 'Male')
df.gender = df.gender.replace(0, 'Unknown')

19. Let's change the birth_year to the 'int64' data type. We only need this column to be in whole numbers, or integers

In [21]:
df.birth_year = df.birth_year.astype('int64')
print(df.dtypes)
print(df.head())

trip_duration_sec     int64
start_time           object
stop_time            object
start_station_id      int64
end_station_id        int64
bike_id               int64
user_type            object
birth_year            int64
gender               object
dtype: object
   trip_duration_sec           start_time            stop_time  \
0                362  2016-01-01 00:02:52  2016-01-01 00:08:54   
1                200  2016-01-01 00:18:22  2016-01-01 00:21:42   
2                202  2016-01-01 00:18:25  2016-01-01 00:21:47   
3                248  2016-01-01 00:23:13  2016-01-01 00:27:21   
6                445  2016-01-01 01:07:45  2016-01-01 01:15:11   

   start_station_id  end_station_id  bike_id   user_type  birth_year  gender  
0              3186            3209    24647  Subscriber        1964  Female  
1              3186            3213    24605  Subscriber        1962    Male  
2              3186            3213    24689  Subscriber        1962  Female  
3              3209  

20. Let's parse out the date only in both the start_time and stop_time columns into new columns start_date and stop_date, respectively.

In [22]:
df['start_date'] = pd.to_datetime(df['start_time']).dt.date
df['end_date'] = pd.to_datetime(df['stop_time']).dt.date
print(df.dtypes)
print(df.head())

trip_duration_sec     int64
start_time           object
stop_time            object
start_station_id      int64
end_station_id        int64
bike_id               int64
user_type            object
birth_year            int64
gender               object
start_date           object
end_date             object
dtype: object
   trip_duration_sec           start_time            stop_time  \
0                362  2016-01-01 00:02:52  2016-01-01 00:08:54   
1                200  2016-01-01 00:18:22  2016-01-01 00:21:42   
2                202  2016-01-01 00:18:25  2016-01-01 00:21:47   
3                248  2016-01-01 00:23:13  2016-01-01 00:27:21   
6                445  2016-01-01 01:07:45  2016-01-01 01:15:11   

   start_station_id  end_station_id  bike_id   user_type  birth_year  gender  \
0              3186            3209    24647  Subscriber        1964  Female   
1              3186            3213    24605  Subscriber        1962    Male   
2              3186            3213    24

20. Now that we have our start_date and stop_date, let's *drop* the start_time and stop_time columns and place the start_date and end_date in their place.

In [25]:
df = df[['trip_duration_sec', 'start_date', 'end_date', 'start_station_id', 'end_station_id', 'bike_id', 'user_type',
        'birth_year', 'gender']]
print(df.head())
print(df.dtypes)

   trip_duration_sec  start_date    end_date  start_station_id  \
0                362  2016-01-01  2016-01-01              3186   
1                200  2016-01-01  2016-01-01              3186   
2                202  2016-01-01  2016-01-01              3186   
3                248  2016-01-01  2016-01-01              3209   
6                445  2016-01-01  2016-01-01              3186   

   end_station_id  bike_id   user_type  birth_year  gender  
0            3209    24647  Subscriber        1964  Female  
1            3213    24605  Subscriber        1962    Male  
2            3213    24689  Subscriber        1962  Female  
3            3203    24693  Subscriber        1984    Male  
6            3203    24510  Subscriber        1988  Female  
trip_duration_sec     int64
start_date           object
end_date             object
start_station_id      int64
end_station_id        int64
bike_id               int64
user_type            object
birth_year            int64
gender       

21. Now both our datasets are clean enough for the database we will create. Let's write these to CSV files.

In [24]:
df.to_csv('trip_data.csv', index=False)
all_stations.to_csv('stations.csv', index=False)